In [1]:
from fastlane_bot import Bot, Config, ConfigDB, ConfigNetwork, ConfigProvider
from fastlane_bot.tools.cpc import ConstantProductCurve as CPC, CPCContainer, T, Pair
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPC))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(Bot))
from fastlane_bot.testing import *
import itertools as it
import collections as cl
plt.style.use('seaborn-dark')
plt.rcParams['figure.figsize'] = [12,6]
from fastlane_bot import __VERSION__
require("3.0", __VERSION__)

Using default database url, if you want to use a different database, set the backend_url found at the bottom of manager_base.py
ConstantProductCurve v2.10.2 (07/May/2023)
CarbonBot v3-b2.1 (03/May/2023)
imported m, np, pd, plt, os, sys, decimal; defined iseq, raises, require
Version = 3-b2.1 [requirements >= 3.0 is met]


# Mainnet Server [A010]

In [2]:
bot     = Bot()
CCm     = bot.get_curves()

In [3]:
pairs0  = CCm.pairs(standardize=False)
pairs   = CCm.pairs(standardize=True)
pairsc  = {c.pairo.primary for c in CCm if c.P("exchange")=="carbon_v1"}

## Overall market 

In [4]:
print(f"Total pairs:    {len(pairs0):4}")
print(f"Primary pairs:  {len(pairs):4}")
print(f"...carbon:      {len(pairsc):4}")
print(f"Tokens:         {len(CCm.tokens()):4}")
print(f"Curves:         {len(CCm):4}")

Total pairs:     665
Primary pairs:   636
...carbon:        25
Tokens:          512
Curves:          811


## By pair 

### All pairs

In [5]:
cbp0 = {pair: [c for c in CCm.bypairs(pair)] for pair in CCm.pairs()} # curves by (primary) pair
nbp0 = {pair: len(cc) for pair,cc in cbp0.items()}
assert len(cbp0) == len(CCm.pairs())
assert set(cbp0) == CCm.pairs()

### Only those with >1 curves

In [6]:
cbp = {pair: cc for pair, cc in cbp0.items() if len(cc)>1}
nbp = {pair: len(cc) for pair,cc in cbp.items()}
print(f"Pairs with >1 curves:  {len(cbp)}")
print(f"Curves in those:       {sum(nbp.values())}")
print(f"Average curves/pair:   {sum(nbp.values())/len(cbp):.1f}")

Pairs with >1 curves:  110
Curves in those:       285
Average curves/pair:   2.6


### x=0 or y=0

In [7]:
xis0 = {c.cid: (c.x, c.y) for c in CCm if c.x==0}
yis0 = {c.cid: (c.x, c.y) for c in CCm if c.y==0}
assert len(xis0) == 0 # set loglevel debug to see removal of curves
assert len(yis0) == 0

### Prices

#### All

In [8]:
prices_da = {pair: 
             [(
                Pair.n(pair), c.primaryp(), c.cid, c.cid[-8:], c.P("exchange"), c.tvl(tkn=pair.split("/")[0]),
                "x" if c.itm(cc) else "", c.buysell(verbose=False), c.buysell(verbose=True, withprice=True)
              ) for c in cc 
             ] 
             for pair, cc in cbp.items()
            }
#prices_da

#### Only for pairs that have at least on Carbon pair

In [9]:
prices_d = {pair: l for pair,l in prices_da.items() if pair in pairsc}
prices_l = list(it.chain(*prices_d.values()))

In [10]:
curves_by_pair = list(cl.Counter([r[0] for r in prices_l]).items())
curves_by_pair = sorted(curves_by_pair, key=lambda x: x[1], reverse=True)
curves_by_pair

[('WETH/USDC', 20),
 ('BNT/WETH', 12),
 ('BNT/vBNT', 10),
 ('USDT/USDC', 8),
 ('WBTC/WETH', 5),
 ('LINK/USDT', 5),
 ('WBTC/USDT', 4),
 ('BNT/USDC', 4),
 ('DAI/USDT', 3),
 ('WETH/DAI', 3),
 ('LINK/USDC', 3),
 ('WETH/USDT', 3),
 ('DAI/USDC', 3),
 ('PEPE/WETH', 3),
 ('WBTC/USDC', 2),
 ('0x0/WETH', 2),
 ('LYXe/USDC', 2),
 ('rETH/WETH', 2),
 ('ARB/MATIC', 2),
 ('TSUKA/USDC', 2),
 ('stETH/WETH', 2)]

In [11]:
# for pair, _ in curves_by_pair:
#     print(f"# #### {pair}\n\npricedf.loc['{pair}']\n\n")

#### Dataframe

In [12]:
pricedf0 = pd.DataFrame(prices_l, columns="pair,price,cid,cid0,exchange,vl,itm,bs,bsv".split(","))
pricedf = pricedf0.drop('cid', axis=1).sort_values(by=["pair", "exchange", "cid0"])
pricedf = pricedf.set_index(["pair", "exchange", "cid0"])
pricedf

price            vl itm  bs  \
pair       exchange     cid0                                          
0x0/WETH   carbon_v1    132277-0     0.000013  7.757952e+03       b   
                        132277-1     0.000014  5.363000e+03       s   
ARB/MATIC  carbon_v1    806240-0     1.507045  1.276054e+01       s   
                        806240-1     1.428571  1.418060e+02       b   
BNT/USDC   bancor_v2    652          0.469913  1.492710e+06   x  bs   
...                                       ...           ...  ..  ..   
WETH/USDT  uniswap_v2   256       1918.786596  3.195897e+04      bs   
rETH/WETH  carbon_v1    903115-0     1.069000  1.870907e+00       b   
           sushiswap_v2 833          1.237861  3.116368e-04      bs   
stETH/WETH carbon_v1    422914-0     1.010101  2.031521e-03       s   
                        422914-1     0.990099  8.011450e-02       b   

                                                                    bsv  
pair       exchange     cid0                                             
0x0/WETH   carbon_v1    132277-0            buy-0x0 @ 0.00 WETH per 0x0  
                        132277-1           sell-0x0 @ 0.00 WETH per 0x0  
ARB/MATIC  carbon_v1    806240-0          sell-ARB @ 1.51 MATIC per ARB  
                        806240-1           buy-ARB @ 1.43 MATIC per ARB  
BNT/USDC   bancor_v2    652            buy-sell-BNT @ 0.47 USDC per BNT  
...                                                                 ...  
WETH/USDT  uniswap_v2   256       buy-sell-WETH @ 1918.79 USDT per WETH  
rETH/WETH  carbon_v1    903115-0          buy-rETH @ 1.07 WETH per rETH  
           sushiswap_v2 833          buy-sell-rETH @ 1.24 WETH per rETH  
stETH/WETH carbon_v1    422914-0       sell-stETH @ 1.01 WETH per stETH  
                        422914-1        buy-stETH @ 0.99 WETH per stETH  

[100 rows x 5 columns]

### Individual frames

#### WETH/USDC

In [13]:
pricedf.loc['WETH/USDC']

price            vl itm  bs  \
exchange     cid0                                          
carbon_v1    057285-1  2099.999790      0.006040       s   
             057292-0  1853.408818      0.003314       b   
             057292-1  2000.000000      0.016387       s   
             057296-0  1929.999824      0.461405   x   b   
             057296-1  1949.999818     10.000019       s   
             057299-1  1940.000000      0.026117       s   
             057306-0  1405.000140      3.558719       b   
             057315-1  2300.000000      0.487950       s   
             057331-0  1800.000000      5.555556       b   
             057334-0  1700.000170      0.029412       b   
             057334-1  1999.999800      0.040000       s   
             057337-0  1850.000000      1.081081       b   
             057339-0  1800.000000      0.000556       b   
             057343-1  1989.999801      1.000000       s   
             057353-0  1854.000185      4.234699       b   
             057353-1  2047.999795      4.000000       s   
sushiswap_v2 803       1918.328630  18000.709598      bs   
uniswap_v2   255       1917.166981  38437.683898      bs   
uniswap_v3   346       1915.241349    223.316190      bs   
             593       1910.636978     22.022255   x  bs   

                                                         bsv  
exchange     cid0                                             
carbon_v1    057285-1      sell-WETH @ 2100.00 USDC per WETH  
             057292-0       buy-WETH @ 1853.41 USDC per WETH  
             057292-1      sell-WETH @ 2000.00 USDC per WETH  
             057296-0       buy-WETH @ 1930.00 USDC per WETH  
             057296-1      sell-WETH @ 1950.00 USDC per WETH  
             057299-1      sell-WETH @ 1940.00 USDC per WETH  
             057306-0       buy-WETH @ 1405.00 USDC per WETH  
             057315-1      sell-WETH @ 2300.00 USDC per WETH  
             057331-0       buy-WETH @ 1800.00 USDC per WETH  
             057334-0       buy-WETH @ 1700.00 USDC per WETH  
             057334-1      sell-WETH @ 2000.00 USDC per WETH  
             057337-0       buy-WETH @ 1850.00 USDC per WETH  
             057339-0       buy-WETH @ 1800.00 USDC per WETH  
             057343-1      sell-WETH @ 1990.00 USDC per WETH  
             057353-0       buy-WETH @ 1854.00 USDC per WETH  
             057353-1      sell-WETH @ 2048.00 USDC per WETH  
sushiswap_v2 803       buy-sell-WETH @ 1918.33 USDC per WETH  
uniswap_v2   255       buy-sell-WETH @ 1917.17 USDC per WETH  
uniswap_v3   346       buy-sell-WETH @ 1915.24 USDC per WETH  
             593       buy-sell-WETH @ 1910.64 USDC per WETH

#### BNT/WETH

In [14]:
pricedf.loc['BNT/WETH']

price            vl itm  bs  \
exchange   cid0                                       
bancor_v2  623       0.000242  1.206183e+07   x  bs   
bancor_v3  704       0.000246  3.125669e+07   x  bs   
carbon_v1  326030-0  0.000253  5.000000e+02       s   
           326030-1  0.000200  2.500000e+02       b   
           326031-0  0.000200  7.499999e+02       b   
           326031-1  0.002000  5.000000e+02       s   
           326034-0  0.000200  3.500000e+02       b   
           326034-1  0.002100  2.000000e+02       s   
           326076-0  0.000253  7.905138e+02   x   b   
           326077-0  0.000250  3.654723e-01   x   b   
           326077-1  0.000258  7.936392e+03       s   
uniswap_v2 290       0.000243  2.137445e+05   x  bs   

                                                  bsv  
exchange   cid0                                        
bancor_v2  623       buy-sell-BNT @ 0.00 WETH per BNT  
bancor_v3  704       buy-sell-BNT @ 0.00 WETH per BNT  
carbon_v1  326030-0      sell-BNT @ 0.00 WETH per BNT  
           326030-1       buy-BNT @ 0.00 WETH per BNT  
           326031-0       buy-BNT @ 0.00 WETH per BNT  
           326031-1      sell-BNT @ 0.00 WETH per BNT  
           326034-0       buy-BNT @ 0.00 WETH per BNT  
           326034-1      sell-BNT @ 0.00 WETH per BNT  
           326076-0       buy-BNT @ 0.00 WETH per BNT  
           326077-0       buy-BNT @ 0.00 WETH per BNT  
           326077-1      sell-BNT @ 0.00 WETH per BNT  
uniswap_v2 290       buy-sell-BNT @ 0.00 WETH per BNT

#### BNT/vBNT

In [15]:
pricedf.loc['BNT/vBNT']

price            vl itm  bs  \
exchange  cid0                                       
bancor_v2 653       1.333016  5.350297e+04   x  bs   
bancor_v3 742       1.252518  2.466230e+06   x  bs   
carbon_v1 748950-0  0.940000  1.413879e+04       b   
          748965-1  0.909091  1.129751e+03       b   
          748966-1  1.000000  1.089256e+03       b   
          748976-0  1.331480  2.593859e+02       s   
          748976-1  1.111111  7.293739e+02       b   
          748977-0  1.428571  5.000000e+02       s   
          748977-1  1.250000  4.000000e+02       b   
          748990-1  0.952381  8.383963e+02       b   

                                                 bsv  
exchange  cid0                                        
bancor_v2 653       buy-sell-BNT @ 1.33 vBNT per BNT  
bancor_v3 742       buy-sell-BNT @ 1.25 vBNT per BNT  
carbon_v1 748950-0       buy-BNT @ 0.94 vBNT per BNT  
          748965-1       buy-BNT @ 0.91 vBNT per BNT  
          748966-1       buy-BNT @ 1.00 vBNT per BNT  
          748976-0      sell-BNT @ 1.33 vBNT per BNT  
          748976-1       buy-BNT @ 1.11 vBNT per BNT  
          748977-0      sell-BNT @ 1.43 vBNT per BNT  
          748977-1       buy-BNT @ 1.25 vBNT per BNT  
          748990-1       buy-BNT @ 0.95 vBNT per BNT

#### USDT/USDC

In [16]:
pricedf.loc['USDT/USDC']

price            vl itm  bs  \
exchange     cid0                                       
carbon_v1    634371-0  1.005025  5.010000e+01       s   
             634371-1  0.995025  5.040025e+01       b   
             634391-0  1.001001  5.050000e+02       s   
             634391-1  1.000690  4.946550e+02       b   
sushiswap_v2 805       0.993768  1.849228e+03      bs   
uniswap_v2   246       1.002118  2.369742e+07      bs   
uniswap_v3   357       1.002654  3.514249e+03      bs   
             486       1.001069  1.011262e+06      bs   

                                                      bsv  
exchange     cid0                                          
carbon_v1    634371-0      sell-USDT @ 1.01 USDC per USDT  
             634371-1       buy-USDT @ 1.00 USDC per USDT  
             634391-0      sell-USDT @ 1.00 USDC per USDT  
             634391-1       buy-USDT @ 1.00 USDC per USDT  
sushiswap_v2 805       buy-sell-USDT @ 0.99 USDC per USDT  
uniswap_v2   246       buy-sell-USDT @ 1.00 USDC per USDT  
uniswap_v3   357       buy-sell-USDT @ 1.00 USDC per USDT  
             486       buy-sell-USDT @ 1.00 USDC per USDT

#### WBTC/WETH

In [17]:
pricedf.loc['WBTC/WETH']

price          vl itm  bs  \
exchange     cid0                                      
carbon_v1    709362-0  15.399750    0.133758       s   
             709362-1  14.285714    0.417087       b   
sushiswap_v2 804       15.099383  776.430469   x  bs   
uniswap_v3   466       15.097158    0.127246   x  bs   
             478       15.312285    0.000571   x  bs   

                                                       bsv  
exchange     cid0                                           
carbon_v1    709362-0      sell-WBTC @ 15.40 WETH per WBTC  
             709362-1       buy-WBTC @ 14.29 WETH per WBTC  
sushiswap_v2 804       buy-sell-WBTC @ 15.10 WETH per WBTC  
uniswap_v3   466       buy-sell-WBTC @ 15.10 WETH per WBTC  
             478       buy-sell-WBTC @ 15.31 WETH per WBTC

#### LINK/USDT

In [18]:
pricedf.loc['LINK/USDT']

price         vl itm  bs  \
exchange     cid0                                    
carbon_v1    960408-0  7.100000  26.783355   x   b   
             960408-1  7.700000  10.874600       s   
sushiswap_v2 791       7.123545  75.100134   x  bs   
uniswap_v2   171       7.328775  65.660409   x  bs   
uniswap_v3   549       6.976809   3.304874   x  bs   

                                                      bsv  
exchange     cid0                                          
carbon_v1    960408-0       buy-LINK @ 7.10 USDT per LINK  
             960408-1      sell-LINK @ 7.70 USDT per LINK  
sushiswap_v2 791       buy-sell-LINK @ 7.12 USDT per LINK  
uniswap_v2   171       buy-sell-LINK @ 7.33 USDT per LINK  
uniswap_v3   549       buy-sell-LINK @ 6.98 USDT per LINK

#### WBTC/USDT

In [19]:
pricedf.loc['WBTC/USDT']

price        vl itm  bs  \
exchange     cid0                                       
carbon_v1    920820-0  28980.881784  0.006836       b   
             920820-1  29500.000000  0.000065       s   
sushiswap_v2 814       29043.429150  0.031191      bs   
uniswap_v2   183       28831.097041  0.336923      bs   

                                                          bsv  
exchange     cid0                                              
carbon_v1    920820-0       buy-WBTC @ 28980.88 USDT per WBTC  
             920820-1      sell-WBTC @ 29500.00 USDT per WBTC  
sushiswap_v2 814       buy-sell-WBTC @ 29043.43 USDT per WBTC  
uniswap_v2   183       buy-sell-WBTC @ 28831.10 USDT per WBTC

#### BNT/USDC

In [20]:
pricedf.loc['BNT/USDC']

price            vl itm  bs  \
exchange  cid0                                       
bancor_v2 652       0.469913  1.492710e+06   x  bs   
bancor_v3 720       0.465186  5.081971e+06   x  bs   
carbon_v1 480199-0  2.000000  2.910000e+01       s   
          480202-1  1.480041  4.247463e+04       s   

                                                 bsv  
exchange  cid0                                        
bancor_v2 652       buy-sell-BNT @ 0.47 USDC per BNT  
bancor_v3 720       buy-sell-BNT @ 0.47 USDC per BNT  
carbon_v1 480199-0      sell-BNT @ 2.00 USDC per BNT  
          480202-1      sell-BNT @ 1.48 USDC per BNT

#### WETH/DAI

In [21]:
pricedf.loc['WETH/DAI']

,,price,vl,itm,bs,bsv
exchange,cid0,,,,,
carbon_v1,211457-1,1944.999806,0.001000,,s,sell-WETH @ 1945.00 DAI per WETH
sushiswap_v2,817,1923.592818,4184.139471,x,bs,buy-sell-WETH @ 1923.59 DAI per WETH
uniswap_v3,594,1902.663470,8.339070,x,bs,buy-sell-WETH @ 1902.66 DAI per WETH


#### LINK/USDC

In [22]:
pricedf.loc['LINK/USDC']

,,price,vl,itm,bs,bsv
exchange,cid0,,,,,
carbon_v1,497903-1,7.750000,342.883761,,s,sell-LINK @ 7.75 USDC per LINK
sushiswap_v2,792,6.935941,3.543424,x,bs,buy-sell-LINK @ 6.94 USDC per LINK
uniswap_v2,176,6.794102,183.739077,x,bs,buy-sell-LINK @ 6.79 USDC per LINK


#### DAI/USDC

In [23]:
pricedf.loc['DAI/USDC']

price            vl itm  bs  \
exchange     cid0                                       
carbon_v1    845828-0  1.001001     30.000000       s   
             845828-1  0.999001     20.019998       b   
sushiswap_v2 839       1.003088  25646.214235      bs   

                                                    bsv  
exchange     cid0                                        
carbon_v1    845828-0      sell-DAI @ 1.00 USDC per DAI  
             845828-1       buy-DAI @ 1.00 USDC per DAI  
sushiswap_v2 839       buy-sell-DAI @ 1.00 USDC per DAI

#### WETH/USDT

In [24]:
pricedf.loc['WETH/USDT']

,,price,vl,itm,bs,bsv
exchange,cid0,,,,,
carbon_v1,691656-0,1900.000190,0.002632,,b,buy-WETH @ 1900.00 USDT per WETH
sushiswap_v2,840,1916.546534,10462.631094,,bs,buy-sell-WETH @ 1916.55 USDT per WETH
uniswap_v2,256,1918.786596,31958.967041,,bs,buy-sell-WETH @ 1918.79 USDT per WETH


#### DAI/USDT

In [25]:
pricedf.loc['DAI/USDT']

price          vl itm  bs  \
exchange     cid0                                     
carbon_v1    268742-0  0.995000   20.100501       b   
             268742-1  1.005000   20.000000       s   
sushiswap_v2 795       0.994182  380.253948      bs   

                                                    bsv  
exchange     cid0                                        
carbon_v1    268742-0       buy-DAI @ 1.00 USDT per DAI  
             268742-1      sell-DAI @ 1.00 USDT per DAI  
sushiswap_v2 795       buy-sell-DAI @ 0.99 USDT per DAI

#### PEPE/WETH

In [26]:
pricedf.loc['PEPE/WETH']

price            vl itm bs  \
exchange  cid0                                          
carbon_v1 440620-0  3.500000e-07  1.454145e+04      b   
          440620-1  4.000000e-07  7.130133e+06      s   
          440621-1  4.500000e-07  1.315789e+06      s   

                                               bsv  
exchange  cid0                                      
carbon_v1 440620-0   buy-PEPE @ 0.00 WETH per PEPE  
          440620-1  sell-PEPE @ 0.00 WETH per PEPE  
          440621-1  sell-PEPE @ 0.00 WETH per PEPE

#### LYXe/USDC

In [27]:
pricedf.loc['LYXe/USDC']

,,price,vl,itm,bs,bsv
exchange,cid0,,,,,
carbon_v1,652071-1,15.999998,7503.700799,,s,sell-LYXe @ 16.00 USDC per LYXe
uniswap_v3,558,14.391360,1.085336,,bs,buy-sell-LYXe @ 14.39 USDC per LYXe


#### rETH/WETH

In [28]:
pricedf.loc['rETH/WETH']

,,price,vl,itm,bs,bsv
exchange,cid0,,,,,
carbon_v1,903115-0,1.069000,1.870907,,b,buy-rETH @ 1.07 WETH per rETH
sushiswap_v2,833,1.237861,0.000312,,bs,buy-sell-rETH @ 1.24 WETH per rETH


#### 0x0/WETH

In [29]:
pricedf.loc['0x0/WETH']

price           vl itm bs                           bsv
exchange  cid0                                                                
carbon_v1 132277-0  0.000013  7757.951901      b   buy-0x0 @ 0.00 WETH per 0x0
          132277-1  0.000014  5363.000000      s  sell-0x0 @ 0.00 WETH per 0x0

#### WBTC/USDC

In [30]:
pricedf.loc['WBTC/USDC']

,,price,vl,itm,bs,bsv
exchange,cid0,,,,,
carbon_v1,537493-0,28000.000000,0.045365,,b,buy-WBTC @ 28000.00 USDC per WBTC
uniswap_v2,332,28931.165371,6.015072,,bs,buy-sell-WBTC @ 28931.17 USDC per WBTC


#### ARB/MATIC

In [31]:
pricedf.loc['ARB/MATIC']

price          vl itm bs                            bsv
exchange  cid0                                                                
carbon_v1 806240-0  1.507045   12.760538      s  sell-ARB @ 1.51 MATIC per ARB
          806240-1  1.428571  141.806023      b   buy-ARB @ 1.43 MATIC per ARB

#### TSUKA/USDC

In [32]:
pricedf.loc['TSUKA/USDC']

,,price,vl,itm,bs,bsv
exchange,cid0,,,,,
carbon_v1,017697-1,0.120000,4.656734e+04,,s,sell-TSUKA @ 0.12 USDC per TSUKA
uniswap_v2,254,0.032085,4.336406e+07,,bs,buy-sell-TSUKA @ 0.03 USDC per TSUKA


#### stETH/WETH

In [33]:
pricedf.loc['stETH/WETH']

price        vl itm bs  \
exchange  cid0                                  
carbon_v1 422914-0  1.010101  0.002032      s   
          422914-1  0.990099  0.080114      b   

                                                 bsv  
exchange  cid0                                        
carbon_v1 422914-0  sell-stETH @ 1.01 WETH per stETH  
          422914-1   buy-stETH @ 0.99 WETH per stETH

In [34]:
raise

RuntimeError: No active exception to reraise

## Execution 

### Configuration

- `flt`: flashloanable tokens
- `loglevel`: `LL_DEBUG` , `LL_INFO` `LL_WARN` `LL_ERR`

In [ ]:
flt = [T.USDC]
C = Config.new(config=Config.CONFIG_TENDERLY, loglevel=Config.LL_INFO)

In [ ]:
bot = CarbonBot(ConfigObj=C)

### Database update [Tenderly specific]

In [ ]:
# provided here for convenience; must be commented out for tests
bot.update(drop_tables=True, top_n=10, only_carbon=False)

### Execution

In [ ]:
bot.run(flashloan_tokens=flt, mode=bot.RUN_SINGLE)

## Execution analysis 

In [ ]:
CCm = bot.get_curves()

### Arbitrage opportunities

In [ ]:
ops = bot._run(flashloan_tokens=flt, CCm=CCm, result=bot.XS_ARBOPPS)
ops

### Route struct

In [ ]:
try:
    route_struct = bot._run(flashloan_tokens=flt, CCm=CCm, result=bot.XS_ROUTE)
except bot.NoArbAvailable as e:
    print(f"[NoArbAvailable] {e}")
    route_struct = None
route_struct

### Orderering info

In [ ]:
try:
    ordinfo = bot._run(flashloan_tokens=flt, CCm=CCm, result=bot.XS_ORDINFO)
    flashloan_amount = ordinfo[1]
    flashloan_token_address = ordinfo[2]
    print(f"Flashloan: {flashloan_amount} [{flashloan_token_address}]")
except bot.NoArbAvailable as e:
    print(f"[NoArbAvailable] {e}")
    ordinfo = None
ordinfo

## Market analysis 

### Overall market

In [ ]:
exch0 = {c.P("exchange") for c in CCm}
print("Number of curves:", len(CCm))
print("Number of tokens:", len(CCm.tokens()))
#print("Exchanges:", exch0)
print("---")
for xc in exch0:
    print(f"{xc+':':16} {len(CCm.byparams(exchange=xc)):4}")

### Pair

In [ ]:
pair = f"{T.ECO}/{T.USDC}"

In [ ]:
CCp = CCm.bypairs(pair)
exch = {c.P("exchange") for c in CCp}
print("pair:           ", pair)
print("curves:         ", len(CCp))
print("exchanges:      ", exch)
for xc in exch:
    c = CCp.byparams(exchange=xc)[0]
    print(f"{xc+':':16} {c.p:.4f} {1/c.p:.4f}")

## Technical 

### Validation and assertions

In [ ]:
assert C.DATABASE == C.DATABASE_POSTGRES
assert C.POSTGRES_DB == "tenderly"
assert C.NETWORK == C.NETWORK_TENDERLY
assert C.PROVIDER == C.PROVIDER_TENDERLY
assert str(type(bot.db)) == "<class 'fastlane_bot.db.manager.DatabaseManager'>"
assert C.w3.provider.endpoint_uri.startswith("https://rpc.tenderly.co/fork/")
assert bot.db.carbon_controller.address == '0xC537e898CD774e2dCBa3B14Ea6f34C93d5eA45e1'

### Tenderly shell commands

Run those commands in a shell if there are Tenderly connection issues

In [ ]:
C_nw = ConfigNetwork.new(network=ConfigNetwork.NETWORK_TENDERLY)
c1, c2 = C_nw.shellcommand().splitlines()
print(c1)
print(c2)
# !{c1}
# !{c2}